<a href="https://colab.research.google.com/github/Thandeka20/High-spatial-resolution-imagery-vs-high-spectral-imagery/blob/main/Code/Sentinel_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import google earth engine API**

In [1]:
import ee

**Authenticate google earth engine and your intialize the cloud project**

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='thandeka-skosana')

**Import libraries needed to run the code**

In [12]:
#import modules
!pip install rasterio
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import geemap
import re
import os
import gdown
import json
from IPython.display import Image, display
from google.colab import drive
from google.colab import files
import shutil
!pip install pycrs
import pycrs
drive.mount('/content/drive')

  Preparing metadata (setup.py) ... done
  Created wheel for pycrs: filename=PyCRS-1.0.2-py3-none-any.whl size=32688 sha256=706caf135502ce7bb7ad8dce6c95cde1b9b48e62507a6836f065ae8d4eeeb7b3
  Stored in directory: /root/.cache/pip/wheels/47/1d/70/7a5bdf33347e7c75e95b06b1fa38f076a59a9506653cc24aff
Successfully built pycrs
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Visualize Sentinel-2 Scene**

In [4]:
#visualize the Sentinel-2 scene
# Define the bands to use for visualization
bands = ee.List(["B4", "B3", "B2"])

# Define visualization parameters
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
}

# Example: Display an image using the defined visualization parameters
image = ee.Image('COPERNICUS/S2/20230727T073621_20230727T075808_T36JTT')
display_image = Image(url=image.visualize(**vis_params).getThumbURL({'dimensions': '500x500'}))
display(display_image)

**Compute vegetation indices**

In [5]:
# Define vegetation indices
GI = image.expression('(GREEN)/(RED)', {
    'GREEN': image.select('B3'),
    'RED': image.select('B4'),
}).rename('GI')

IRG = image.expression('(RED-GREEN)', {
    'RED': image.select('B4'),
    'GREEN': image.select('B3'),
}).rename('IRG')

NGRDI = image.expression('(GREEN-RED)/(GREEN+RED)', {
    'RED': image.select('B4'),
    'GREEN': image.select('B3'),
}).rename('NGRDI')

VARI = image.expression('(GREEN-RED)/(GREEN+RED+BLUE)', {
    'RED': image.select('B3'),
    'GREEN': image.select('B4'),
    'BLUE': image.select('B2'),
}).rename('VARI')

VDVI = image.expression('(2*GREEN-RED-BLUE)/(2*GREEN+RED+BLUE)', {
    'RED': image.select('B4'),
    'GREEN': image.select('B3'),
    'BLUE': image.select('B2'),
}).rename('VDVI')

WBI = image.expression('(Blue - Red) / (Blue + Red)', {
    'Blue': image.select('B2'),
    'Red': image.select('B4'),
}).rename('WBI')

# Add computed variables as bands to the image_final
image_final = image.addBands([GI, IRG, NGRDI, VARI, VDVI, WBI])
# Print image bands
print(image_final.getInfo())

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [1830, 1830], 'crs': 'EPSG:32736', 'crs_transform': [60, 0, 199980, 0, -60, 7300000]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32736', 'crs_transform': [10, 0, 199980, 0, -10, 7300000]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32736', 'crs_transform': [10, 0, 199980, 0, -10, 7300000]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32736', 'crs_transform': [10, 0, 199980, 0, -10, 7300000]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [5490, 5490], 'crs': 'EPSG:32736', 'crs_transform': [20, 0, 199980, 0, -20, 7300000

**Load training data from google drive to google colab**

In [13]:
#load training data
file_path = '/content/drive/My Drive/Colab Notebooks/Data/TrainingSet_SabieCroc/Training_setSc.shp'
training_data = geemap.shp_to_ee(file_path)
print(training_data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



**Sample regions from the image at training data locations**

In [14]:
#select bands
bands= ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8','B8A', 'B9', 'B11', 'B12','GI', 'IRG', 'NGRDI', 'VARI', 'VDVI', 'WBI',];

# Sample regions from the image at training feature locations
training_dataset = image_final.select(bands).sampleRegions(
    collection=training_data,
    properties=['ID', 'LULC_Class'],
    scale=10
)


**Run the classification using random forest**

In [15]:
#Classification
classifier = ee.Classifier.smileRandomForest(100).train(
    features=training_dataset,
    classProperty='ID',
    inputProperties=bands
)
classified = image_final.select(bands).classify(classifier)


**Defining a colour palette for the classified map and viweing the map**

In [18]:
# Define the palette
SabieCrocPalette = [
    '351C75', 'F91DF9', '980A7D', '741b47', 'fd0618', 'E06666', 'ffcc99', 'ffffff', '999999', 'a8a800',
    '6aa84f', '14870e', 'DB992D', 'ff7f00', '000000', '0a14f9', '08f3e4'
]

# Visualization parameters
viz = {
    'min': 1,
    'max': 17,
    'palette': SabieCrocPalette
}

# Create a Map object using geemap
m = geemap.Map(center=[20, 0], zoom=3)
print (m)

# Add the classified image to the map
m.addLayer(classified, viz, 'Classification')
m.centerObject(classified, zoom=10)

# Display the legend
#m.add_legend(title='Classification', colors=SabieCrocPalette, labels=[str(i) for i in range(1, 18)])

# Display the map
m




Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(children=(ToggleButton(value=False, icon='globe', layout=Layout(height='28px', padding='0px 0px 0px 4px', width='28px'), tooltip='Search location/data'),), _dom_classes=('geemap-colab',))), ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text', 'zoom_out_title']), FullScreenControl(options=['position']), MapDrawControl(marker={'shapeOptions': {'color': '#3388ff'}}, options=['position'], polygon={'shapeOptions': {}}, polyline={'shapeOptions': {}}, rectangle={'shapeOptions': {'color': '#3388ff'}}), ScaleControl(options=['imperial', 'max_width', 'metric', 'position', 'update_when_idle'], position='bottomleft'), MeasureControl(active_color='orange', options=['active_color', 'capture_z_index', 'completed_color', 'popup_options', 'position', 'primary_area_unit', 'primary_length_unit', 'secondary_area_unit', 'secondary_length_unit'], popup_options={'className': 

Map(center=[-24.91471009556816, 30.694320050468605], controls=(WidgetControl(options=['position', 'transparent…